In [1]:
import pandas as pd
import geopandas as gpd
import shapely.geometry as geom
import numpy as np
import json

In [2]:

gdf : gpd.GeoDataFrame = gpd.read_file('local/MB_2021_AUST_SHP_GDA2020/MB_2021_AUST_GDA2020.shp')
# 4m

In [3]:
gdf : gpd.GeoDataFrame = gdf[['MB_CODE21', 'geometry']].rename(columns={'MB_CODE21': 'id'})

In [4]:
gdf_geom : gpd.GeoSeries = gdf.boundary
# 30s

In [5]:
# Combine multiple linestring and multilinestring geometries into single multilinestring geometry
gdml = gdf_geom.apply(lambda x: geom.MultiLineString([x]) if isinstance(x, geom.LineString) else x)
# 1m

In [6]:
gl = gdml.apply(lambda x: x.geoms if x is not None else None).explode()
# 1m

In [7]:
gl.dropna(inplace=True)

In [8]:
coords = gl.apply(lambda x: np.array(x.coords))
# 1m 30s for list, 10s for numpy array

In [9]:
coords.reset_index(drop=True, inplace=True)

In [10]:
coords

0         [[146.92785052248806, -36.083647638487946], [1...
1         [[146.93543157545233, -36.04752350882983], [14...
2         [[146.93040435597695, -36.049962708462175], [1...
3         [[146.9325386953542, -36.0523838486409], [146....
4         [[146.9304386362757, -36.04733293844756], [146...
                                ...                        
371519    [[167.994230239959, -29.045246369874825], [167...
371520    [[150.69854153401263, -35.136031388427355], [1...
371521    [[105.66649220160672, -10.422030053283393], [1...
371522    [[96.88759755192176, -12.110863535868376], [96...
371523    [[167.97281153877768, -29.016557751834803], [1...
Length: 371524, dtype: object

In [11]:
# Get all points
points = np.concatenate(coords.to_numpy())
points.sort(axis=0)
len(points)
# 19647448
# 10s

19647448

In [12]:
# Drop duplicates
points = np.unique(points, axis=0)
len(points)
# 14983534

14983534

In [14]:
# Split long and lat into separate columns, and save to csv, with index and header
df = pd.DataFrame(points, columns=['long', 'lat'])
df.to_csv('local/mb_points.csv', index=True, header=True)
# 1m 30s